# <center>The Smith Parasite - An Unknown Parasitic Disease</center>

A new disease has recently been discovered by Dr. Smith, in England. You have been brought in to investigate.
The disease has already affected more than 5000 people, with no apparent connection between them.

The most common symptoms include fever and tiredness, but some infected people are asymptomatic. Regardless, this virus is being associated with post-disease conditions such as loss of speech, confusion, chest pain and shortness of breath.

The conditions of the transmission of the disease are still unknown and there are no certainties of what leads a patient to suffer or not from it. Nonetheless, some groups of people seem more prone to be infected by the parasite than others.

In this challenge, your goal is to build a predictive model that answers the question, “Who are the people more likely to suffer from the Smith Parasite?”. With that goal, you can access a small quantity of sociodemographic, health, and behavioral information obtained from the patients.

As data scientists, your team is asked to analyze and transform the data available as needed and apply different models to answer the defined question in a more accurate way. Can you build a model that can predict if a patient will suffer, or not, from the Smith Disease?

---

### Datasets
The training set should be used to build your machine learning models. In this set, you also have the ground truth associated to each patient, i.e., if the patient has the disease (Disease = 1) or not (Disease = 0).

The test set should be used to see how well your model performs on unseen data. In this set you don’t have access to the ground truth, and the goal of your team is to predict that value (0 or 1) by using the model you created using the training set.

The score of your predictions is the percentage of instances you correctly predict, using the **f1 score**.

---

### Deliverables

You should submit a csv file with the number of instances in the test set, containing the columns [PatientID, Disease], and only those columns. <br>
The Disease column should contain the prediction (“0” or “1”). Your last solution (the Jupiter notebook) should be submitted on Moodle.

---

In [ ]:
import os
import pandas as pd
import numpy as np
import functools
import math
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, learning_curve, RandomizedSearchCV, StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import RocCurveDisplay, roc_auc_score, f1_score, classification_report

In [ ]:
data = {dataset_name.replace('.xlsx', ''): pd.read_excel(f'Data/{dataset_name}') for dataset_name in os.listdir('Data') if 'xlsx' in dataset_name}        

In [ ]:
print(list(data.keys()))

In [ ]:
data_frames = [data['train_demo'], data['train_habits'], data['train_health']]
df = functools.reduce(lambda  left,right: pd.merge(left,right,on=['PatientID'], how='outer'), data_frames).set_index('PatientID')

In [ ]:
data_frames = [data['test_demo'], data['test_habits'], data['test_health']]
df_test = functools.reduce(lambda  left,right: pd.merge(left,right,on=['PatientID'], how='outer'), data_frames).set_index('PatientID')

---
### Exploratory Data Analysis

In [ ]:
df.head(5)

In [ ]:
df.dtypes

In [ ]:
pd.concat([df.isnull().sum(),df.eq('').sum()],keys=['Nulls','Empty Strings'],axis=1)

In [ ]:
df.duplicated().sum()

In [ ]:
df.describe(include="all").T

In [ ]:
sns.set()

sns.pairplot(df, hue = 'Disease', markers = ['o', 's'])

In [ ]:
df_numeric_features = list(df.select_dtypes(include = np.number).columns)
df_numeric_features.remove('Disease')

fig, ax = plt.subplots(math.ceil(len(df_numeric_features)/4),4, figsize = (15,10))
for ax, feat in zip(ax.flatten(), df_numeric_features):
    ax.boxplot(df[feat], notch = True, patch_artist = True)
    ax.set_title(feat)

plt.show()

In [ ]:
df[df['Birth_Year'] < 1961]['Birth_Year'].value_counts()

### Takeaways from EDA
- Regarding the numeric features "Physical_Health" does have the highest correlation
- There're outliers in the "Birth_Year" column. Probably not possible?

---
### Baseline
- only numeric features

In [ ]:
x, y = df.drop(columns = ['Disease']), df['Disease']

In [ ]:
xnumeric = x.select_dtypes(include = np.number)

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(xnumeric, y, random_state = 111 ,test_size = 0.3, shuffle = True , stratify = y)

In [ ]:
model = LogisticRegression(max_iter = 500)
model.fit(xtrain,ytrain)

ypred = model.predict(xval)

print(classification_report(yval, ypred))

---
### Preprocessing
- Impute
- One-Hot-Encoding
- Scale

Regarding the Birth_Year column and it's strange outliers (e.g. 1869) we have several options.<br>
Considering we're using the **quantile +/- 1.5 * interquantile range** or IQR method rule we only end up with 12 critical observations

1. The easy way would be to just filter out the observation. In that case every observation below 1941 would get dropped
2. Another way would be to just fill those few observation with the mode
3. The last option and probably the best would be to assume 1869 to be a typo. We could change all Birth_Year observations from the 19th century to the 20th century - 1869 --> 1969

In [ ]:
def preprocessing(dataframe):

    # fix Birth_Year errors (1869 > 1969)
    dataframe['Birth_Year'] = [i + 100 if i < 1900 else i for i in dataframe['Birth_Year']]

    # add Age column
    dataframe['Age'] = [2022 - i for i in dataframe['Birth_Year']]

    # add Gender column (1: Male, 0: Female)
    # afterwards drop column Name
    dataframe['Gender']  = [1 if i.split(' ')[0] == 'Mr.' else 0 for i in dataframe['Name']]
    dataframe.drop(columns = ['Name'], inplace = True)

    # add column population density
    dataframe['Region'] = [i.lower() for i in dataframe['Region']]
    dataframe['Region_Density'] = [dataframe['Region'].value_counts(normalize = True)[i] for i in dataframe['Region']]

    # encode Smoking_Habit & Exercise to binary (1: Yes, 0: No)
    dataframe['Smoking_Habit'] = [1 if i == 'Yes' else 0 for i in dataframe['Smoking_Habit']]
    dataframe['Exercise'] = [1 if i == 'Yes' else 0 for i in dataframe['Exercise']]

    # try knn imputing?
    # impute missing values in column "Education" with mode
    dataframe['Education'].fillna(dataframe['Education'].mode()[0], inplace=True)

    # split x in numeric and categorical features
    xnumeric, xcategorical = dataframe.select_dtypes(include = np.number), dataframe.select_dtypes(exclude = np.number)

    # One-hot-encoding categorical features
    encoder = OneHotEncoder(handle_unknown='ignore')
    x_cat_encoded = pd.DataFrame(encoder.fit_transform(xcategorical).toarray(), columns = encoder.get_feature_names_out(), index = xcategorical.index) 

    # Scale numeric features
    scaler = MinMaxScaler(feature_range=(-1, 1))
    x_num_scaled = pd.DataFrame(scaler.fit_transform(xnumeric), index = xnumeric.index ,columns = xnumeric.columns)

    xpreprocessed = pd.concat([x_num_scaled, x_cat_encoded], axis = 1)

    return xpreprocessed

In [ ]:
x, y = df.drop(columns = ['Disease']), df['Disease']

In [ ]:
xpreprocessed = preprocessing(x)

y = y.loc[xpreprocessed.index]

---
### Train, test and validation split

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(xpreprocessed, y, random_state = 222 , test_size = 0.3, shuffle = True , stratify = y)
xval, xtest, yval, ytest = train_test_split(xval, yval, random_state = 333, test_size = 0.5, shuffle = True, stratify = yval)

In [ ]:
model = LogisticRegression(max_iter = 500)
model.fit(xtrain,ytrain)

ypred = model.predict(xval)

print(classification_report(yval, ypred))

---

### Feature Selection

##### Correlation Matrix - here we're able to just LabelEncode our categorical variables

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

xnumeric, xcategorical = x.select_dtypes(include = np.number), x.select_dtypes(exclude = np.number)
encoder = OrdinalEncoder()
x_cat_encoded = pd.DataFrame(encoder.fit_transform(xcategorical), index = xcategorical.index, columns = xcategorical.columns)

xcorrelation = pd.concat([xnumeric, x_cat_encoded], axis = 1)

In [ ]:
correlation = pd.concat([xcorrelation, y], axis = 1).corr(method = 'spearman')

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(correlation, dtype=bool))

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(20, 15))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(correlation, annot = True, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.show()

In [ ]:
model = RandomForestClassifier(random_state = 444)
rfe = RFECV(model, cv = StratifiedKFold(10), scoring = 'f1')
rfe.fit(xtrain, ytrain)
opt_features = list(rfe.get_feature_names_out(input_features = list(xtrain.columns)))
print(f'Best features: {[i for i in opt_features]}')

In [ ]:
xtrain_rfe = rfe.transform(xtrain)
xval_rfe = rfe.transform(xval)

---
### Model Selection

In [ ]:
def roc_auc(classifiers, xtrain, ytrain, xval, yval):
    '''
    Plots ROC/AUC
    classifiers input --> {'Logistic Regression': LogisticRegression(),...}
    '''

    fig, ax = plt.subplots(1, figsize=(15, 10))
    for name, clf in classifiers.items():
        clf.fit(xtrain, ytrain)
        RocCurveDisplay.from_estimator(clf, xval, yval, ax=ax, name=name)
    ax.set_title('Receiver Operating Characteristic (ROC)')
    ax.plot([0,1], [0,1], linestyle='--')
    plt.show()

classifiers = {
                'Logistic Regression': LogisticRegression(),
                'SVM': SVC(),
                'Neural Network': MLPClassifier(max_iter = 500),
                'Decision Tree': DecisionTreeClassifier(),
                'Random Forest': RandomForestClassifier(),
                'GradientBoostingClassifier': GradientBoostingClassifier()
              }

In [ ]:
roc_auc(classifiers, xtrain_rfe, ytrain, xval_rfe, yval)

In [ ]:
def learning_curves(estimator, X, Y):
    '''
    Plots learning curve
    '''

    train_sizes, train_scores, validation_scores = learning_curve(estimator, X, Y, cv = 10, scoring = 'f1', train_sizes = np.arange(.05,1,.05))
    train_mean, test_mean, train_std, test_std = np.mean(train_scores, axis=1), np.mean(validation_scores, axis=1), np.std(train_scores, axis=1), np.std(validation_scores, axis=1)

    plt.subplots(1, figsize=(10,10))
    plt.plot(train_sizes, train_mean, color='salmon',  label='Training score', marker = 'o')
    plt.plot(train_sizes, test_mean, color='olive', label='Cross-validation score', marker = 's')
    plt.title('Learning Curve')
    plt.xlabel('Training Set Size')
    plt.ylabel('F1 Score')
    plt.legend(loc='best')
    plt.show()

learning_curves(DecisionTreeClassifier(random_state = 555), xpreprocessed, y)

---
### Hyperparamter tuning

- GridSearch

In [ ]:
param_grid = {
                'bootstrap': [True, False],
                'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
                'max_features': ['auto', 'sqrt'],
                'min_samples_leaf': [1, 2, 4],
                'min_samples_split': [2, 5, 10],
                'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
            }


model = RandomForestClassifier(random_state = 666)
rand_search = RandomizedSearchCV(estimator = model, param_distributions = param_grid, scoring = 'f1', n_iter = 100, cv = 3, verbose=2, random_state=777, n_jobs = -1)
rand_search.fit(xtrain_rfe, ytrain)

print(str(rand_search.best_params_).replace('{','').replace('}','').replace("'","").replace(':','='))

---
### Train model with best classifier, optimized feature selection and hyperparameter tuning

In [ ]:
model = RandomForestClassifier(**rand_search.best_params_)
model.fit(xtrain_rfe, ytrain)

validation_pred = model.predict(xval_rfe)

print(classification_report(yval, validation_pred))

---
### Final train with all of our available data

In [ ]:
xfull = xpreprocessed

xfull_rfe = rfe.transform(xfull)

model = RandomForestClassifier(**rand_search.best_params_)
model.fit(xfull_rfe, y)

---
### Predict test dataset

In [ ]:
xtest = df_test

xtest_preprocessed = preprocessing(xtest)

xtest_rfe = rfe.transform(xtest_preprocessed)

In [ ]:
ypred = model.predict(xtest_rfe)

---
### Create CSV-file for submission

In [ ]:
# df_submission = pd.concat([pd.Series(xtest.index),pd.Series(ypred)], axis = 1)
# df_submission.rename(columns = {0:'Disease'}, inplace = True)
# df_submission

In [ ]:
# df_submission.to_csv('Group01_Version03.csv', index = False)